In [1]:
import pandas as pd
import numpy as np

In [9]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/teams/v1/international"

headers = {
    "x-rapidapi-key": "ba1d0f0236mshc26046c1f2ad005p146cf7jsnb965cc8f2c2e",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

teams = response.json()

In [10]:
teams

{'list': [{'teamName': 'Test Teams'},
  {'teamId': 2,
   'teamName': 'India',
   'teamSName': 'IND',
   'imageId': 719031,
   'countryName': 'India'},
  {'teamId': 96,
   'teamName': 'Afghanistan',
   'teamSName': 'AFG',
   'imageId': 172188},
  {'teamId': 27, 'teamName': 'Ireland', 'teamSName': 'IRE', 'imageId': 495001},
  {'teamId': 3,
   'teamName': 'Pakistan',
   'teamSName': 'PAK',
   'imageId': 591986,
   'countryName': 'Pakistan'},
  {'teamId': 4,
   'teamName': 'Australia',
   'teamSName': 'AUS',
   'imageId': 172117,
   'countryName': 'Australia'},
  {'teamId': 5,
   'teamName': 'Sri Lanka',
   'teamSName': 'SL',
   'imageId': 172119,
   'countryName': 'Sri Lanka'},
  {'teamId': 6,
   'teamName': 'Bangladesh',
   'teamSName': 'BAN',
   'imageId': 172120},
  {'teamId': 9, 'teamName': 'England', 'teamSName': 'ENG', 'imageId': 172123},
  {'teamId': 10,
   'teamName': 'West Indies',
   'teamSName': 'WI',
   'imageId': 172124},
  {'teamId': 11,
   'teamName': 'South Africa',
   'te

In [104]:
teams_df = pd.json_normalize(teams,record_path=["list"])

In [105]:
teams_data = teams_df.loc[1:12,["teamId","teamName"]]

In [106]:
teams_data["teamId"] = teams_data["teamId"].astype(int)

In [107]:
teams_data

,teamId,teamName
1,2,India
2,96,Afghanistan
3,27,Ireland
4,3,Pakistan
5,4,Australia
6,5,Sri Lanka
7,6,Bangladesh
8,9,England
9,10,West Indies
10,11,South Africa


In [74]:
team_players = pd.DataFrame()
for idx in teams_data["teamId"]:
    import requests

    url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{idx}/players"

    headers = {
        "x-rapidapi-key": "ba1d0f0236mshc26046c1f2ad005p146cf7jsnb965cc8f2c2e",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers)

    players = response.json()
    dup = pd.json_normalize(players,record_path=["player"])
    dup["teamId"] = idx
    team_players = pd.concat([team_players,dup])
    
team_players

,name,imageId,id,battingStyle,bowlingStyle,teamId
0,BATSMEN,174146,NaN,NaN,NaN,2
1,Shubman Gill,616515,11808,Right-hand bat,Right-arm offbreak,2
2,Yashasvi Jaiswal,591942,13940,Left-hand bat,Right-arm legbreak,2
3,Sai Sudharsan,717782,13866,Left-hand bat,Right-arm legbreak,2
4,Rohit Sharma,616514,576,Right-hand bat,Right-arm offbreak,2
...,...,...,...,...,...,...
24,Jacob Duffy,616417,8554,Right-hand bat,Right-arm fast-medium,13
25,Matt Henry,616431,9067,Right-hand bat,Right-arm fast-medium,13
26,Kyle Jamieson,616419,9441,Right-hand bat,Right-arm fast-medium,13
27,William ORourke,616423,15925,Right-hand bat,Right-arm fast-medium,13


In [75]:
team_players = team_players.loc[:,["name","id","battingStyle","bowlingStyle","teamId"]]
team_players

,name,id,battingStyle,bowlingStyle,teamId
0,BATSMEN,NaN,NaN,NaN,2
1,Shubman Gill,11808,Right-hand bat,Right-arm offbreak,2
2,Yashasvi Jaiswal,13940,Left-hand bat,Right-arm legbreak,2
3,Sai Sudharsan,13866,Left-hand bat,Right-arm legbreak,2
4,Rohit Sharma,576,Right-hand bat,Right-arm offbreak,2
...,...,...,...,...,...
24,Jacob Duffy,8554,Right-hand bat,Right-arm fast-medium,13
25,Matt Henry,9067,Right-hand bat,Right-arm fast-medium,13
26,Kyle Jamieson,9441,Right-hand bat,Right-arm fast-medium,13
27,William ORourke,15925,Right-hand bat,Right-arm fast-medium,13


In [91]:
final = pd.merge(teams_data,team_players,on="teamId")

In [93]:
ff = final.dropna(axis="index",thresh=4,ignore_index=True)

In [102]:
ff

,teamId,teamName,name,id,battingStyle,bowlingStyle
0,2,India,Shubman Gill,11808,Right-hand bat,Right-arm offbreak
1,2,India,Yashasvi Jaiswal,13940,Left-hand bat,Right-arm legbreak
2,2,India,Sai Sudharsan,13866,Left-hand bat,Right-arm legbreak
3,2,India,Rohit Sharma,576,Right-hand bat,Right-arm offbreak
4,2,India,Virat Kohli,1413,Right-hand bat,Right-arm medium
...,...,...,...,...,...,...
349,13,New Zealand,Jacob Duffy,8554,Right-hand bat,Right-arm fast-medium
350,13,New Zealand,Matt Henry,9067,Right-hand bat,Right-arm fast-medium
351,13,New Zealand,Kyle Jamieson,9441,Right-hand bat,Right-arm fast-medium
352,13,New Zealand,William ORourke,15925,Right-hand bat,Right-arm fast-medium


In [40]:
ff.to_csv("teams_dataset.csv")

NameError: name 'ff' is not defined

In [41]:
f = pd.read_csv("teams_dataset")

In [59]:
f.to_csv("players.csv")

In [53]:
main_data = pd.DataFrame()
for idx in test_data["id"]:
    import requests

    url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{int(idx)}/batting"

    headers = {
        "x-rapidapi-key": "ba1d0f0236mshc26046c1f2ad005p146cf7jsnb965cc8f2c2e",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers)

    batting = response.json()
    final = pd.DataFrame(batting["headers"])
    for i in batting["values"]:
        x = i["values"]
        y = pd.DataFrame(x)
        final = pd.concat([final,y],axis=1)
    final.columns=final.iloc[0]
    final = final.drop(index=0).reset_index(drop=True)
    final["id"] = idx
    main_data = pd.concat([main_data,final])

In [54]:
main_data

,ROWHEADER,Matches,Innings,Runs,Balls,Highest,Average,SR,Not Out,Fours,Sixes,Ducks,50s,100s,200s,300s,400s,id
0,Test,34,63,2478,3951,269,42.72,62.72,5,273,43,5,7,8,1,0,0,11808
1,ODI,55,55,2775,2787,208,59.04,99.57,8,314,59,1,15,8,1,0,0,11808
2,T20,21,21,578,415,126,30.42,139.28,2,60,22,1,3,1,0,0,0,11808
3,IPL,118,115,3866,2787,129,39.45,138.72,17,372,119,4,26,4,0,0,0,11808
0,Test,21,40,2018,3037,214,53.11,66.45,2,243,40,3,11,5,2,0,0,13940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,IPL,32,5,10,18,6,10,55.56,4,0,0,0,0,0,0,0,0,10754
0,Test,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12926
1,ODI,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12926
2,T20,18,3,1,6,1,0.5,16.67,1,0,0,2,0,0,0,0,0,12926


In [57]:
main_data.to_csv("india_batters.csv")

In [2]:
ibts = pd.read_csv("india_batters.csv")
pl = pd.read_csv("players.csv")

In [63]:
ibts

,Unnamed: 0,ROWHEADER,Matches,Innings,Runs,Balls,Highest,Average,SR,Not Out,Fours,Sixes,Ducks,50s,100s,200s,300s,400s,id
0,0,Test,34,63,2478,3951,269,42.72,62.72,5,273,43,5,7,8,1,0,0,11808
1,1,ODI,55,55,2775,2787,208,59.04,99.57,8,314,59,1,15,8,1,0,0,11808
2,2,T20,21,21,578,415,126,30.42,139.28,2,60,22,1,3,1,0,0,0,11808
3,3,IPL,118,115,3866,2787,129,39.45,138.72,17,372,119,4,26,4,0,0,0,11808
4,0,Test,21,40,2018,3037,214,53.11,66.45,2,243,40,3,11,5,2,0,0,13940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,3,IPL,32,5,10,18,6,10.00,55.56,4,0,0,0,0,0,0,0,0,10754
144,0,Test,0,0,0,0,0,0.00,0.00,0,0,0,0,0,0,0,0,0,12926
145,1,ODI,4,0,0,0,0,0.00,0.00,0,0,0,0,0,0,0,0,0,12926
146,2,T20,18,3,1,6,1,0.50,16.67,1,0,0,2,0,0,0,0,0,12926


In [64]:
pl

,Unnamed: 0.1,Unnamed: 0,teamId,teamName,name,id,battingStyle,bowlingStyle
0,0,0,2,India,Shubman Gill,11808,Right-hand bat,Right-arm offbreak
1,1,1,2,India,Yashasvi Jaiswal,13940,Left-hand bat,Right-arm legbreak
2,2,2,2,India,Sai Sudharsan,13866,Left-hand bat,Right-arm legbreak
3,3,3,2,India,Rohit Sharma,576,Right-hand bat,Right-arm offbreak
4,4,4,2,India,Virat Kohli,1413,Right-hand bat,Right-arm medium
...,...,...,...,...,...,...,...,...
349,349,349,13,New Zealand,Jacob Duffy,8554,Right-hand bat,Right-arm fast-medium
350,350,350,13,New Zealand,Matt Henry,9067,Right-hand bat,Right-arm fast-medium
351,351,351,13,New Zealand,Kyle Jamieson,9441,Right-hand bat,Right-arm fast-medium
352,352,352,13,New Zealand,William ORourke,15925,Right-hand bat,Right-arm fast-medium


In [66]:
pd.merge(pl,ibts,on="id")

,Unnamed: 0.1,Unnamed: 0_x,teamId,teamName,name,id,battingStyle,bowlingStyle,Unnamed: 0_y,ROWHEADER,...,SR,Not Out,Fours,Sixes,Ducks,50s,100s,200s,300s,400s
0,0,0,2,India,Shubman Gill,11808,Right-hand bat,Right-arm offbreak,0,Test,...,62.72,5,273,43,5,7,8,1,0,0
1,0,0,2,India,Shubman Gill,11808,Right-hand bat,Right-arm offbreak,1,ODI,...,99.57,8,314,59,1,15,8,1,0,0
2,0,0,2,India,Shubman Gill,11808,Right-hand bat,Right-arm offbreak,2,T20,...,139.28,2,60,22,1,3,1,0,0,0
3,0,0,2,India,Shubman Gill,11808,Right-hand bat,Right-arm offbreak,3,IPL,...,138.72,17,372,119,4,26,4,0,0,0
4,1,1,2,India,Yashasvi Jaiswal,13940,Left-hand bat,Right-arm legbreak,0,Test,...,66.45,2,243,40,3,11,5,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,35,35,2,India,Mukesh Kumar,10754,Right-hand bat,Right-arm fast-medium,3,IPL,...,55.56,4,0,0,0,0,0,0,0,0
144,36,36,2,India,Varun Chakaravarthy,12926,Right-hand bat,Right-arm legbreak,0,Test,...,0.00,0,0,0,0,0,0,0,0,0
145,36,36,2,India,Varun Chakaravarthy,12926,Right-hand bat,Right-arm legbreak,1,ODI,...,0.00,0,0,0,0,0,0,0,0,0
146,36,36,2,India,Varun Chakaravarthy,12926,Right-hand bat,Right-arm legbreak,2,T20,...,16.67,1,0,0,2,0,0,0,0,0


In [2]:
main_data = pd.DataFrame()
for idx in test_data["id"]
    import requests

    url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{int(idx)}/bowling"

    headers = {
        "x-rapidapi-key": "ba1d0f0236mshc26046c1f2ad005p146cf7jsnb965cc8f2c2e",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers)

    bowling = response.json()
    final = pd.DataFrame(bowling["headers"])
    for i in bowling["values"]:
        x = i["values"]
        y = pd.DataFrame(x)
        final = pd.concat([final,y],axis=1)
    final.columns=final.iloc[0]
    final = final.drop(index=0).reset_index(drop=True)
    final["id"] = idx
    main_data = pd.concat([main_data,final])

In [5]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/12926/bowling"

headers = {
    "x-rapidapi-key": "ba1d0f0236mshc26046c1f2ad005p146cf7jsnb965cc8f2c2e",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }
    
response = requests.get(url, headers=headers)

bowling = response.json()
final = pd.DataFrame(bowling["headers"])
for i in bowling["values"]:
    x = i["values"]
    y = pd.DataFrame(x)
    final = pd.concat([final,y],axis=1)
final.columns=final.iloc[0]
final = final.drop(index=0).reset_index(drop=True)
final["id"] = 12926

In [6]:
final

,ROWHEADER,Matches,Innings,Balls,Runs,Maidens,Wickets,Avg,Eco,SR,BBI,BBM,4w,5w,10w,id
0,Test,0,0,0,0,0,0,0.0,0,0.0,-/-,-/-,0,0,0,12926
1,ODI,4,4,240,190,0,10,19.0,4.75,24.0,5/42,5/42,0,1,0,12926
2,T20,18,18,411,481,0,33,14.58,7.02,12.45,5/17,5/17,0,2,0,12926
3,IPL,83,83,1888,2385,1,100,23.85,7.58,18.88,5/20,5/20,1,1,0,12926


In [7]:
yashwanth = pl.loc[37:136,:]

In [8]:
yashwanth.to_csv("y_data.csv")

In [24]:
bhavya = pl.loc[137:236,:]

In [25]:
bhavya.to_csv("b_data.csv")

In [9]:
vignesh = pl.loc[237:336,:]

In [28]:
vignesh.to_csv("v_data.csv")

In [14]:
d = {"headers":["reowheadr","ipl",'t20',"test"],
    "values":{"values":["matches"],
             "values":["inn"],
             "values":["run"]}}

In [20]:
pd.json_normalize(d,rec)

,headers,values.values
0,"[reowheadr, ipl, t20, test]",[run]


In [19]:
for i in d["headers"]:
    print(i)

reowheadr
ipl
t20
test


In [6]:
pd.DataFrame(d["headers"])

,0
0,reowheadr
1,ipl
2,t20
3,test


In [6]:
yashwanth

,Unnamed: 0.1,Unnamed: 0,teamId,teamName,name,id,battingStyle,bowlingStyle
37,37,37,96,Afghanistan,Hashmatullah Shahidi,8277,Left-hand bat,Right-arm offbreak
38,38,38,96,Afghanistan,Rahmat Shah,8571,Right-hand bat,Right-arm legbreak
39,39,39,96,Afghanistan,Riaz Hassan,14200,Right-hand bat,NaN
40,40,40,96,Afghanistan,Sediqullah Atal,14204,Left-hand bat,NaN
41,41,41,96,Afghanistan,Bahir Shah,15753,Right-hand bat,Right-arm offbreak
...,...,...,...,...,...,...,...,...
133,133,133,3,Pakistan,Shaheen Afridi,12160,Left-hand bat,Left-arm fast-medium
134,134,134,3,Pakistan,Mohammad Hasnain,14248,Right-hand bat,Right-arm fast-medium
135,135,135,4,Australia,Steven Smith,2250,Right-hand bat,Right-arm legbreak
136,136,136,4,Australia,Marnus Labuschagne,10182,Right-hand bat,Right-arm legbreak


In [4]:
d = {"headers":["reowheadr","ipl",'t20',"test"],
    "values":["values":["matches"],
             "values":["inn"],
             "values":["run"]]}

SyntaxError: invalid syntax (4215851467.py, line 2)

In [5]:
import pandas as pd
import numpy as np

In [6]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/12926/bowling"

headers = {
    "x-rapidapi-key": "ba1d0f0236mshc26046c1f2ad005p146cf7jsnb965cc8f2c2e",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }
    
response = requests.get(url, headers=headers)

bowling = response.json()
final = pd.DataFrame(bowling["headers"])
for i in bowling["values"]:
    x = i["values"]
    y = pd.DataFrame(x)
    final = pd.concat([final,y],axis=1)
final.columns=final.iloc[0]
final = final.drop(index=0).reset_index(drop=True)
final["id"] = 12926

KeyError: 'headers'

In [9]:
x = pd.read_json("all_players_batting_career_data.json")

In [21]:
data["values"]

[{'values': ['Matches', '34', '55', '21', '118']},
 {'values': ['Innings', '63', '55', '21', '115']},
 {'values': ['Runs', '2478', '2775', '578', '3866']},
 {'values': ['Balls', '3951', '2787', '415', '2787']},
 {'values': ['Highest', '269', '208', '126', '129']},
 {'values': ['Average', '42.72', '59.04', '30.42', '39.45']},
 {'values': ['SR', '62.72', '99.57', '139.28', '138.72']},
 {'values': ['Not Out', '5', '8', '2', '17']},
 {'values': ['Fours', '273', '314', '60', '372']},
 {'values': ['Sixes', '43', '59', '22', '119']},
 {'values': ['Ducks', '5', '1', '1', '4']},
 {'values': ['50s', '7', '15', '3', '26']},
 {'values': ['100s', '8', '8', '1', '4']},
 {'values': ['200s', '1', '1', '0', '0']},
 {'values': ['300s', '0', '0', '0', '0']},
 {'values': ['400s', '0', '0', '0', '0']}]

In [24]:
final = pd.DataFrame()
for i in data["values"]:
    x= pd.DataFrame(i["values"])
    final = pd.concat([final,x],axis=1)

In [25]:
final

,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Matches,Innings,Runs,Balls,Highest,Average,SR,Not Out,Fours,Sixes,Ducks,50s,100s,200s,300s,400s
1,34,63,2478,3951,269,42.72,62.72,5,273,43,5,7,8,1,0,0
2,55,55,2775,2787,208,59.04,99.57,8,314,59,1,15,8,1,0,0
3,21,21,578,415,126,30.42,139.28,2,60,22,1,3,1,0,0,0
4,118,115,3866,2787,129,39.45,138.72,17,372,119,4,26,4,0,0,0


In [26]:
len(data)

4

14